# Facial Recognition Project using Siamese Neural Networks

## Importing Dependencies

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import random

# tensorflow dependencies
from keras.models import Sequential, Model
from keras.layers import Layer, Dense, Conv2D, Flatten, Input, MaxPool2D
import tensorflow as tf

2024-09-30 16:50:17.432755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 16:50:17.613130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 16:50:17.641063: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 16:50:17.645908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 16:50:17.734712: I tensorflow/core/platform/cpu_feature_guar

## Setting up required files

In [13]:
# setup file paths
pos_path = os.path.join('facial recognition data', 'positive')
neg_path = os.path.join('facial recognition data', 'negative')
anchor_path = os.path.join('facial recognition data', 'anchors')


In [3]:
# uncompress the image dataset for negatives
# from lfw dataset: http://vis-ww.cs.umass.edu/lfw/
 
!tar -xf lfw.tgz

In [4]:
# move lfw images to the correct directories

# iterating over all directories in lfw (contains names and each contains certain images of that person)
for directory in os.listdir('lfw'):
  # iterating over all the files in any given person's folder
  for file in os.listdir((os.path.join('lfw', directory))):
    # defining previous path
    prev_path = os.path.join('lfw', directory, file)
    # creating new path
    new_path = os.path.join(neg_path, file)
    # replacing old path by new path, effectively moving the files
    os.replace(prev_path, new_path)

# might seem new, but is actually pretty intuitive and easy

In [5]:
# Import uuid library to generate unique image names for the images we'll create
import uuid

## Creating and storing the dataset

In [14]:
# Create a connection to the webcam
capture = cv2.VideoCapture(0)
while capture.isOpened():
  ret, frame = capture.read()

  # cut frame down to 250x250 pixels
  frame = frame[50:50+250, 200:200+250]

  # collect anchor images
  if cv2.waitKey(1) & 0XFF == ord("a"):
    # create unique file path
    img_name = os.path.join(anchor_path, '{}.jpg'.format(uuid.uuid1()))
    # save image
    cv2.imwrite(img_name, frame)

  # collect positive imagesqq
  if cv2.waitKey(1) & 0XFF == ord("p"):
    # create unique file path
    img_name = os.path.join(pos_path, '{}.jpg'.format(uuid.uuid1()))
    # save image
    cv2.imwrite(img_name, frame)
    
  # show image in screen
  cv2.imshow("Image Collection", frame)
  if cv2.waitKey(1) & 0XFF == ord("q"):
    # waitKey(1) refreshes the window after every millisecond and returns a 32-bit integer
    # we use & to retain only the required 8 digits (0XFF is 11111111) ord() returns unicode and that is within 0-255
    # so pressing q will break
    break

capture.release()
cv2.destroyAllWindows()

## Preprocessing

In [15]:
# with all the data collected, we can begin preprocessing

def preprocess(img_file_path):
    # Read in image from file path
    byte_img = tf.io.read_file(img_file_path)
    # convert the image to a uint8 tensor
    img = tf.io.decode_jpeg(byte_img)

    # resize the image to 100x100
    img = tf.image.resize(img, (100,100))
    # scale the values
    img /= 255
    return img

In [16]:
# get image directories

anchor = tf.data.Dataset.list_files(anchor_path+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(pos_path+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(neg_path+'\*.jpg').take(300)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: facial recognition data/anchors\\*.jpg'

In [ ]:
# testing if data is loaded fine or not
dir_test = anchor.as_numpy_iterator()
print(dir_test.next())

In [10]:
# Create labelled dataset
positives = tf.data.Dataset.zip((anchor, positive))
y_positives=tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))
negatives = tf.data.Dataset.zip((anchor, negative))
y_negatives = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))
data = positives.concatenate(negatives)
labels = y_positives.concatenate(y_negatives)

In [ ]:
# verify
samples = data.as_numpy_iterator()
print(samples.next())

In [12]:
# write a new function which does the preeprocessing for a column
def preprocess_twin(t, v):
    return (preprocess(t), preprocess(v))

In [13]:
# pipeline to preprocess the data

data = data.map(preprocess_twin)
dataset = tf.data.Dataset.zip((data, labels))
dataset = dataset.cache()     # to load the files onto a cache for faster access
dataset = dataset.shuffle(buffer_size=1024)

In [14]:
# Partitioning into test and train

# Training Partition
data_train = dataset.take(round(len(data)*0.75))
# Make batches of 16
data_train = data_train.batch(16)
# Preprocess 8 batches while previous ones are being trained
data_train = data_train.prefetch(8)

In [15]:
# Testing Partition
data_test = dataset.skip(round(len(data)*0.75))
data_test = data_test.batch(16)
data_test = data_test.prefetch(8)

## Building the Model

In [16]:
# defining  the distance layer

class L1Dist(Layer):
    def __init__(self, **kwargs):
        # kwargs are there to allow any other parameters such as input_dim, units etc
        # see keras documentation of creating a custom layer
        super().__init__()
    
    # the call function of the class is what the model inhernetly uses
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
# defining the cnn embeddings

cnn = Sequential([
    Conv2D(filters=64, kernel_size=(10,10), activation='relu', input_shape=(100,100,3)),
    MaxPool2D(64, (2,2), padding='same'),

    Conv2D(filters=128, kernel_size=(7,7), activation='relu'),
    MaxPool2D(64, (2,2), padding='same'),

    Conv2D(filters=128, kernel_size=(4,4), activation='relu'),
    MaxPool2D(64, (2,2), padding='same'),

    Conv2D(filters=256, kernel_size=(4,4), activation='relu'),

    Flatten(),
    Dense(4096, activation='sigmoid')
])

# input for the cnn
input_image = Input(name='input_img', shape=(100,100,3))
validation_image = Input(name='validation_img', shape=(100,100,3))

# creating the siamese layer
distance_layer = L1Dist()
distances = distance_layer(cnn(input_image), cnn(validation_image))

# creating the final layer
output = Dense(1, activation='sigmoid')(distances)

# Defining the model
model = Model(inputs=[input_image, validation_image], outputs=output, name='siameseNetwork')

In [ ]:
model.summary()

## Compiling the Model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training and Testing

In [ ]:
model.fit(data_train, batch_size=16, epochs=3, verbose=1)

In [ ]:
# accuracy for the test data
loss, accuracy = model.evaluate(data_test)
print(accuracy)

In [ ]:
model.save("face recog model")

In [ ]:
model = tf.keras.models.load_model('face recog model')

## Visualising Results

In [25]:
run_data = data_test.as_numpy_iterator()

In [ ]:
img_input, img_val = run_data.next()[0]
y_hat = model.predict([img_input, img_val])

In [ ]:
y_hat

In [ ]:
# post-processing
[1 if prediction > 0.5 else 0 for prediction in y_hat]

In [ ]:
plt.figure(figsize=(18,8))
plt.subplot(1,2,1)
plt.imshow(img_input[8])
plt.subplot(1,2,2)
plt.imshow(img_val[8])